In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import optuna
from sklearn.metrics import mean_squared_error
import xgboost as xgb

cat_types = ["model", "brand", "ext_col", "int_col", "accident", 
             "clean_title", "body_style",
             'engine','fuel_type']
df = pd.read_csv('cars_train_enriched_acc_noassumption.csv')
df['miles_per_year'] = df['milage']
df['miles_per_year'] = df.apply(lambda x: x['miles_per_year'] / x['age'] if x['age']>0 else 0, axis=1)
df = df.astype({col: "category" for col in cat_types})

In [10]:
drop_cols = ['id', 'price_diff', 'adjusted_price_diff', 'transmission', 'full_name', 'brand_model']
y = df['price']
X = df.drop(['price'], axis=1).drop(drop_cols, axis=1)
feature_importances = []
threshold_opt_cats = ["model", "ext_col", "accident", 
             "clean_title", "body_style",
             'engine','fuel_type', 'int_col', 'brand']
def objective(trial):
    params = {
        'price_threshold': trial.suggest_int('price_threshold', 100000, 3000000, log=True),
        'assume_accident': trial.suggest_int('assume_accident', 0, 1),
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 50, 1200),
        'eta':trial.suggest_float('eta', 0.0001, 0.5, log = True),  # learning rate
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 100, log = True),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
        'lambda': trial.suggest_float('lambda', 0.1, 1000, log = True),
        'alpha': trial.suggest_float('alpha', 1e-4, 100, log = True),
        'tree_method': 'hist',  
        'device':'cpu'
    }
    threshold = {cat:trial.suggest_int(f'{cat}_threshold', 1, 2000) for cat in threshold_opt_cats}
    include_col = {col:trial.suggest_int(f'include_{col}', 0, 1) for col in X.columns}
    
    errors = []
    n=5
    kf = KFold(n_splits=n, shuffle = True, random_state = 1219)
    kf.get_n_splits(X)
    for train_index, valid_index in kf.split(X):
        #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15)
        X_train = X.loc[train_index, :]
        y_train = y[train_index]
        X_valid = X.loc[valid_index, :]
        y_valid = y[valid_index]
        
        X_train = X_train[y_train<params['price_threshold']]
        y_train = y_train[y_train<params['price_threshold']]

        if params['assume_accident']==1: 
            X_train.loc[X_train['accident'] == 'unknown', 'accident'] = 'atleast1accidentordamagereported'
            X_valid.loc[X_valid['accident'] == 'unknown', 'accident'] = 'atleast1accidentordamagereported'

        for cat in cat_types:
            value_counts = X_train[cat].value_counts().to_dict()
            X_train[cat] = X_train[cat].apply(lambda x: x if value_counts[x] > threshold[cat] else "unknown")
            X_valid[cat] = X_valid[cat].apply(lambda x: x if (x in value_counts) and (value_counts[x] > threshold[cat]) else "unknown")

        X_train = X_train[[col for col in include_col if include_col[col]==1]]
        X_valid = X_valid[[col for col in include_col if include_col[col]==1]]
        
        if len(X_train.columns)==0: return 10000000000   #really large number for the case where we drop all columns

        X_train = X_train.astype({col: "category" for col in cat_types if col in X_train.columns})
        X_valid = X_valid.astype({col: "category" for col in cat_types if col in X_valid.columns})

        dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
        dvalid = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)

        # Train the model
        model = xgb.train(params, dtrain, evals=[(dvalid, 'validation')], num_boost_round=params['n_estimators'], early_stopping_rounds=35, verbose_eval=False)
        feature_importances.append(model.get_score(importance_type='gain'))  # get feature importance
        # Predict on the validation set
        y_pred_valid = model.predict(dvalid)

        # Calculate RMSE on the validation set
        rmse = mean_squared_error(y_valid, y_pred_valid, squared=False)
        errors.append(rmse)
    
    return sum(errors)/n



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)


best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

print("Parameter importance:\n", optuna.importance.get_param_importances(study))


[I 2024-09-26 20:48:32,682] A new study created in memory with name: no-name-8d515808-b1de-4950-9a15-85c13374240f
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:48:32] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:48:38] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:48:43] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:50:03] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:50:06] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:50:08] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-26 20:50:10,826] Trial 3 finished with value: 78134.59497093948 and parameters: {'price_threshold': 2

[I 2024-09-26 20:51:57,953] Trial 6 finished with value: 76792.38489804904 and parameters: {'price_threshold': 1560620, 'assume_accident': 1, 'n_estimators': 687, 'eta': 0.0004640130564654129, 'max_depth': 5, 'min_child_weight': 0.0007411188568038045, 'subsample': 0.25242605002534874, 'colsample_bytree': 0.9218306637049928, 'lambda': 665.0253919143936, 'alpha': 0.49064461224791567, 'model_threshold': 1358, 'ext_col_threshold': 1537, 'accident_threshold': 1340, 'clean_title_threshold': 62, 'body_style_threshold': 159, 'engine_threshold': 272, 'fuel_type_threshold': 1125, 'int_col_threshold': 580, 'brand_threshold': 298, 'include_brand': 0, 'include_model': 0, 'include_model_year': 1, 'include_milage': 1, 'include_fuel_type': 0, 'include_engine': 0, 'include_ext_col': 0, 'include_int_col': 1, 'include_accident': 1, 'include_clean_title': 0, 'include_body_style': 0, 'include_msrp': 1, 'include_age': 0, 'include_reliability': 0, 'include_adjusted_msrp': 1, 'include_miles_per_year': 0}. Bes

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:53:38] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:53:41] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:53:44] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:53:47] WARNING: C:\b\abs_90_bwj

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:55:08] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:55:14] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-26 20:55:19,907] Trial 13 finished with value: 72787.6041076538 and parameters: {'price_threshold': 832095, 'assume_accident': 1, 'n_estimators': 374, 'eta': 0.010970294586603814, 'max_depth': 8, 'min_child_weight': 47.299758918286614, 'subsample': 0.6146359821332805, 'colsample_bytree': 0.7837508461434985, 'lambda': 0.9978850473764659, 'alpha': 0.002738796756130554, 'model_threshold': 1502, 'e

...

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:05] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:11] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:19] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:28] WARNING: C:\b\abs_90_bwj

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:39:21] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:39:26] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-26 23:39:32,544] Trial 188 finished with value: 72632.69317187327 and parameters: {'price_threshold': 1665724, 'assume_accident': 0, 'n_estimators': 1012, 'eta': 0.025694533643516464, 'max_depth': 6, 'min_child_weight': 66.95553472516313, 'subsample': 0.2821772953033018, 'colsample_bytree': 0.7514071833795254, 'lambda': 0.1688823108282674, 'alpha': 0.0005479157836499205, 'model_threshold': 578,

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:22] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:26] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:31] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:41:36] WARNING: C:\b\abs_90_bwj

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:43:47] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:43:53] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "assume_accident", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-09-26 23:44:00,312] Trial 195 finished with value: 72606.71924569672 and parameters: {'price_threshold': 1556722, 'assume_accident': 0, 'n_estimators': 951, 'eta': 0.01804493860540013, 'max_depth': 6, 'min_child_weight': 22.608696547808545, 'subsample': 0.2752733709809438, 'colsample_bytree': 0.7915325216561243, 'lambda': 0.19223044856169258, 'alpha': 0.0005221704979269993, 'model_threshold': 661,

KeyboardInterrupt: 

In [11]:
best_params = {'price_threshold': 1194471, 'assume_accident': 0, 'n_estimators': 951, 'eta': 0.013253672942340268, 'max_depth': 6, 'min_child_weight': 38.26739593627571, 'subsample': 0.20646752099818685, 'colsample_bytree': 0.787577422310926, 'lambda': 0.30415017474541467, 'alpha': 0.00030546530869858226, 'model_threshold': 627, 'ext_col_threshold': 1679, 'accident_threshold': 62, 'clean_title_threshold': 917, 'body_style_threshold': 542, 'engine_threshold': 49, 'fuel_type_threshold': 1492, 'int_col_threshold': 1832, 'brand_threshold': 113, 'include_brand': 1, 'include_model': 1, 'include_model_year': 0, 'include_milage': 0, 'include_fuel_type': 1, 'include_engine': 1, 'include_ext_col': 1, 'include_int_col': 1, 'include_accident': 1, 'include_clean_title': 0, 'include_body_style': 1, 'include_msrp': 1, 'include_age': 1, 'include_reliability': 0, 'include_adjusted_msrp': 0, 'include_miles_per_year': 1}
best_params['objective'] = 'reg:squarederror'
best_params['eval_metric'] = 'rmse'
best_params['device'] = 'cpu'
not_included = [col for col in X.columns if best_params[f'include_{col}']==0]

drop_cols_train = ['id', 'price_diff', 'adjusted_price_diff', 'transmission', 'full_name', 'brand_model']
y = df['price']
X = df.drop(['price'], axis=1).drop(drop_cols_train, axis=1)

X = X[y<best_params['price_threshold']]
y = y[y<best_params['price_threshold']]

drop_cols_test = ['transmission', 'full_name', 'brand_model']
dt = pd.read_csv('cars_test_enriched_acc_noassumption.csv')
dt['miles_per_year'] = dt['milage']
dt['miles_per_year'] = dt.apply(lambda x: x['miles_per_year'] / x['age'] if x['age']>0 else 0, axis=1)

if best_params['assume_accident']==1: 
    X.loc[X['accident'] == 'unknown', 'accident'] = 'atleast1accidentordamagereported'
    dt.loc[dt['accident'] == 'unknown', 'accident'] = 'atleast1accidentordamagereported'




for cat in cat_types:
        value_counts = X[cat].value_counts().to_dict()
        X[cat] = X[cat].apply(lambda x: x if value_counts[x] > best_params[f'{cat}_threshold'] else "unknown")
        dt[cat] = dt[cat].apply(lambda x: x if (x in value_counts) and (value_counts[x] > best_params[f'{cat}_threshold']) else "unknown")

X = X.astype({col: "category" for col in cat_types})
dt = dt.astype({col: "category" for col in cat_types})

X.drop(not_included, axis = 1, inplace= True)
dt.drop(drop_cols_test, axis=1, inplace=True)
dt.drop(not_included, axis=1, inplace=True)



dtrain = xgb.DMatrix(X, label=y, enable_categorical=True)
model = xgb.train(best_params, dtrain, num_boost_round=best_params['n_estimators'])

C:\Users\Baraa\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:46:15] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "accident_threshold", "assume_accident", "body_style_threshold", "brand_threshold", "clean_title_threshold", "engine_threshold", "ext_col_threshold", "fuel_type_threshold", "include_accident", "include_adjusted_msrp", "include_age", "include_body_style", "include_brand", "include_clean_title", "include_engine", "include_ext_col", "include_fuel_type", "include_int_col", "include_milage", "include_miles_per_year", "include_model", "include_model_year", "include_msrp", "include_reliability", "int_col_threshold", "model_threshold", "n_estimators", "price_threshold" } are not used.

  warnings.warn(smsg, UserWarning)


In [12]:
ids = dt.pop('id')
dtest = xgb.DMatrix(dt, enable_categorical=True)  
pred = model.predict(dtest)



# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': ids,
    'price': pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission15.csv', index=False)